In [1]:
import pandas as pd
from river import datasets
from deep_river import regression
from torch import nn
from river import compose
from river import preprocessing
from itertools import islice
from sklearn import metrics

In [2]:
class MyModule(nn.Module):
    def __init__(self, n_features):
        super(MyModule, self).__init__()
        self.dense0 = nn.Linear(n_features, 5)
        self.nonlin = nn.ReLU()
        self.dense1 = nn.Linear(5, 1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.nonlin(self.dense1(X))
        X = self.softmax(X)
        return X


def batcher(iterable, batch_size):
    iterator = iter(iterable)
    while batch := list(islice(iterator, batch_size)):
        yield batch

In [3]:
dataset = datasets.Bikes()

model_pipeline = compose.Select(
    "clouds", "humidity", "pressure", "temperature", "wind"
)
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= regression.Regressor(
    module=MyModule, loss_fn="mse", optimizer_fn="sgd"
)
model_pipeline

Pipeline (
  Select (
    clouds
    humidity
    pressure
    temperature
    wind
  ),
  StandardScaler (
    with_std=True
  ),
  Regressor (
    module=None
    loss_fn="mse"
    optimizer_fn="sgd"
    lr=0.001
    is_feature_incremental=False
    device="cpu"
    seed=42
  )
)

In [4]:
y_trues = []
y_preds = []
for batch in batcher(dataset.take(5000), 5):
    x, y = zip(*batch)
    x = pd.DataFrame(x)
    y_trues.extend(y)
    y_preds.extend(model_pipeline.predict_many(X=x))
    model_pipeline.learn_many(X=x, y=y)

/Users/cedrickulbach/Documents/Environments/deep-river/lib/python3.12/site-packages/river/preprocessing/scale.py:244: RuntimeWarning: invalid value encountered in scalar power
  stds = np.array([self.vars[c] ** 0.5 for c in X.columns], dtype=dtype)
/Users/cedrickulbach/Documents/Environments/deep-river/lib/python3.12/site-packages/river/preprocessing/scale.py:244: RuntimeWarning: invalid value encountered in scalar power
  stds = np.array([self.vars[c] ** 0.5 for c in X.columns], dtype=dtype)
/Users/cedrickulbach/Documents/Environments/deep-river/lib/python3.12/site-packages/river/preprocessing/scale.py:244: RuntimeWarning: invalid value encountered in scalar power
  stds = np.array([self.vars[c] ** 0.5 for c in X.columns], dtype=dtype)
/Users/cedrickulbach/Documents/Environments/deep-river/lib/python3.12/site-packages/river/preprocessing/scale.py:244: RuntimeWarning: invalid value encountered in scalar power
  stds = np.array([self.vars[c] ** 0.5 for c in X.columns], dtype=dtype)
/Use

In [5]:
metrics.mean_squared_error(y_true=y_trues, y_pred=y_preds)

102.4412